In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# load data

In [2]:
!ls -l '/content/drive/My Drive/Colab Notebooks/datasets/hap_ang_neu.zip'

-rw------- 1 root root 677300671 Nov  4 07:55 '/content/drive/My Drive/Colab Notebooks/datasets/hap_ang_neu.zip'


In [3]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/datasets/hap_ang_neu.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_╡╡╜╔ ╚п░ц_20210203115848-008-010.jpg  
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_╡╡╜╔ ╚п░ц_20210203115848-008-016.jpg  
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_╗є╛ў╜├╝│_┴б╞ў_╜├└х_20210203115746-003-002.jpg  
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_╗є╛ў╜├╝│_┴б╞ў_╜├└х_20210203115746-003-004.jpg  
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_╝ў╣┌ ╣╫ ░┼┴╓░°░г_20210203115903-010-010.jpg  
  inflating: train/happy/1f830bb52573ddb5f8913e0c233cb34b2847050021b5fe8e4e91f3f2f121e3c5_│▓_20_▒т╗▌_┐└╢Ї_░°┐м╜├╝│_20210203115821-005-002.jpg  
  inflating: train/happy/1ffd138266d307392fb0576b43b11edaca8d7aaded0a9578862547c686ec1012_│▓_50

In [4]:
!ls -l

total 16
drwx------ 5 root root 4096 Nov  5 01:45 drive
drwxr-xr-x 1 root root 4096 Nov  1 13:35 sample_data
drwxr-xr-x 5 root root 4096 Nov  5 01:46 train
drwxr-xr-x 5 root root 4096 Nov  5 01:46 valid


In [5]:
train_dir = 'train'
valid_dir = 'valid'

# build model

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.RMSprop(learning_rate=0.1),
              metrics = ['accuracy'])

In [21]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 148, 198, 32)      896       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 74, 99, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 72, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 34, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 15, 21, 128)      

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,200),
    batch_size=30,
    class_mode = 'categorical')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size = (150, 200),
    batch_size = 30,
    class_mode = 'categorical')

Found 6000 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.


In [22]:
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=20,
    validation_data=valid_generator,
    validation_steps=50)

Epoch 1/20
200/200 [==============================] - 103s 508ms/step - loss: 164969408.0000 - accuracy: 0.3278 - val_loss: 99615.9766 - val_accuracy: 0.3333
Epoch 2/20
200/200 [==============================] - 100s 501ms/step - loss: 787.2825 - accuracy: 0.3363 - val_loss: 1.1610 - val_accuracy: 0.3333
Epoch 3/20
200/200 [==============================] - 101s 504ms/step - loss: 1.1116 - accuracy: 0.3348 - val_loss: 1.1050 - val_accuracy: 0.3333
Epoch 4/20
200/200 [==============================] - 102s 508ms/step - loss: 1.1060 - accuracy: 0.3393 - val_loss: 1.1074 - val_accuracy: 0.3333
Epoch 5/20
200/200 [==============================] - 101s 503ms/step - loss: 1.1039 - accuracy: 0.3390 - val_loss: 1.1107 - val_accuracy: 0.3333
Epoch 6/20
200/200 [==============================] - 101s 505ms/step - loss: 1.1074 - accuracy: 0.3200 - val_loss: 1.1045 - val_accuracy: 0.3333
Epoch 7/20
200/200 [==============================] - 101s 504ms/step - loss: 1.1052 - accuracy: 0.3365 - val_